# Molecular Dynamical Simulations 

## Mathematical Introduction

Newton's Second Law of kinematic motion can be given as follows:

\begin{aligned}
m\frac{d^2r(t)}{dt^2} = F = - \nabla{U(r)}
\end{aligned}

where $m$ is the mass, $r$ is the position, $F$ is the force and $\nabla U(r)$ is the potential energy, which depends only on the position of the body. If one knows the forces acting upon the body, one can find the position of the body at any moment $r(t)$, i.e. predict its dynamics. This can be done by solving Newton’s equation of motion. It is a second order ODE that can be solved analytically for a few simple cases: constant force, harmonic oscillator, periodic force, drag force, etc. However, a more general approach is to use computers in order to solve the ODE numerically.

The second order ODE is transformed to the system of two first order ODEs as follows:

\begin{aligned}
\frac{dr(t)}{dt} = v(t)\\
m\frac{dv(t)}{dt} = F(t)
\end{aligned}

We use a finite difference approximation that comes to a simple forward Euler Algorithm:
\begin{aligned}
v_{n+1} = v_n + \frac{F_n}{m} dt\\
r_{n+1} = r_n + v_{n+1} dt
\end{aligned}
Here we discretize time t with time step $dt$, such that, $t_{n+1}=t_n+dt$ when $r_n=r(t_n),\,v_n=v(t_n)$ where $n$ is the given timestep. 

### Computation

We want to know the dynamics of a green apple (m=0.3 kg) tossed horizontally at 10 cm/s speed from the top of the Toronto CN Tower (553 m) for the first 10 seconds.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(8,8))

## Adjust axes limits according to your problem. Here we don't need more than a couple of meters left or right, and 600 meters up
ax.set(xlim=(-2, 2), ylim=(0, 600), xlabel='Position, meters', ylabel='Height, meters', title='Apple falling from CN tower')

# parameters of the problem
T = 10. #s
m = 0.3 #kg
g = 9.8 #m/s^2
v0x = -0.1 #m/s
H = 553. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem
v = np.zeros((N+1, 2))
r = np.zeros((N+1, 2))
f = np.zeros((N+1, 2))

# initial conditions:
r[0] = np.array([0., H])
v[0] = np.array([-v0x, 0.])

# the only force is gravity
f[:] = np.array([0., -m * g])

## Run dynamics:
for n in range(N):
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing the first data point  
scat = ax.scatter(r[0,0], r[0,1], marker='o', c='g', s=200)

## animating 
def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, func=animate, frames=N)
## this function will create a lot of *.png files in a folder 'CNtower_frames'
## and create an HTML page with a simulation
ani.save('CNtower.html', writer=animation.HTMLWriter(fps= 1//dt))
plt.close()
#ani.save('CNtower.mp4', fps= 1//dt)

In [2]:
from IPython.display import HTML
HTML('CNtower.html')

Following the initial problem statement, let us examine a closed system of classical particles. When a closed system of classical particles are interacting through pairwise potentials, the force on each particle, $i$, depends on its position with respect to every other particle $j$:

\begin{aligned}
m_i\frac{d^2r_i(t)}{dt^2} = \sum_jF_{ij}(t) = -\sum_j\nabla_i{U(|r_{ij}(t)|)}
\end{aligned}

Where $r_{ij} = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2}$ is the distance between particle $i$ and $j$, and $i,j \in (1,N)$.

### 3-Body Simulation under Hooke's Law

We want to know the dynamics of 3 particles m=1kg connected to each other with invisible springs with $K_s = 5$n/m where $K_s$ is Hooke's constant, and $r_0 =1$m initially located at the position vectors (0, 2), (2, 0) and (-1, 0) on the 2D plane for the first 10 seconds of their motion.

The pairwise potential given for Hooke's Law is as follows: 
$U(r_{ij}) = \frac{K_s}{2}(r_{ij} - r_0)^2$

The negative gradient of the potential is a force from $j'th$ upon $i'th$ is given by:

\begin{aligned}
\mathbf{F_{ij}} = - \nabla_i{U(r_{ij})} = - K_s (r_{ij} - r_0) \nabla_i r_{ij} = - K_s (r_{ij} - r_0) \mathbf{r_{ij}} / r_{ij}
\end{aligned}

In [3]:
# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(6,6))
ax.set(xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), ylabel='meters', xlabel='meters', title='3-Body problem')

# parameters of the problem
T = 10. #s
m = 1.0 #kg
ks = 5 #N/m
r0 = 1. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem: first axis - time. second axis - particle's number. third - coordinate
v = np.zeros((N+1, 3, 2))
r = np.zeros((N+1, 3, 2))
f = np.zeros((N+1, 3, 2))

# initial conditions for 3 particles:
r[0,0] = np.array([0., 2.])
r[0,1] = np.array([2., 0.])
r[0,2] = np.array([-1., 0.])

def compute_forces(n):
    '''The function computes forces on each pearticle at time step n'''
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[n,i] - r[n,j]
                rij_abs = np.linalg.norm(rij)
                f[n, i] -= ks * (rij_abs - r0) * rij / rij_abs 
## Run dynamics:
for n in range(N):
    compute_forces(n)
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing and animating 
scat = ax.scatter(r[0,:,0], r[0,:,1], marker='o', c=['b', 'k', 'r'], s=1000)

def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, animate, frames=N)
plt.close()
## this function will create a lot of *.png files in a folder '3Body_frames'
ani.save('3body.html', writer=animation.HTMLWriter(fps= 1//dt))

In [4]:
HTML('3body.html')